In [1]:
from datasets import load_dataset
from tqdm import tqdm

In [2]:
!nvidia-smi

Fri May 27 14:59:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.23       Driver Version: 511.23       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8     4W /  N/A |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
train_dataset  = load_dataset('squad_v2', split="train")
valid_dataset = load_dataset('squad_v2', split="validation")

Reusing dataset squad_v2 (C:\Users\Mapraw\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)
Reusing dataset squad_v2 (C:\Users\Mapraw\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


In [4]:
mini_train_data = train_dataset.select(range(1000))

In [5]:
mini_train_data

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 1000
})

In [6]:
ans_text = [i["text"][0] for i in mini_train_data["answers"]]

In [7]:
mini_train_data = mini_train_data.add_column("ans_text", ans_text)

Loading cached processed dataset at C:\Users\Mapraw\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d\cache-2d829cf492b2b792.arrow


In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
# tokenizer_LM = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")
# model_LM = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum",torch_dtype=torch.float16)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [9]:
# model_LM

In [10]:
import torch

In [11]:
# pt_model_LM = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum",torch_dtype=torch.float16)

In [12]:
# pt_model_LM

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (

In [13]:
model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [14]:
def get_question(answer, context, max_length=64):
  input_text = "answer: %s context: %s </s>" % (answer, context)
#   input_text = "%s [SEP] %s" % (answer, context)
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return tokenizer.decode(output[0])
context = "Manuel has created RuPERTa-base with the support of HF-Transformers and Google"
answer = "Manuel"

get_question(answer, context)

'<pad> question: Who created the RuPERTa-base?</s>'

In [15]:
get_question(mini_train_data["answers"][3], mini_train_data["context"][3])

'<pad> question: Where was Beyonce born?</s>'

In [16]:
input_texts = ["answer: "+ mini_train_data["ans_text"][ind] +"  context: "+ mini_train_data["context"][ind]+" </s>" for ind,_ in enumerate(mini_train_data)]

In [17]:
mini_train_data = mini_train_data.add_column("input_texts", input_texts)

In [18]:
def tokenize_function(examples):
    model_input = tokenizer(examples["input_texts"], truncation=True, padding="max_length")
    label = tokenizer(examples["question"], truncation=True, padding="max_length")
    model_input["labels"] = label["input_ids"]
    return model_input

In [19]:
train_ds = mini_train_data.map(tokenize_function, batched=True)

Loading cached processed dataset at C:\Users\Mapraw\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d\cache-d2fdd2c88d3c4013.arrow


In [20]:
train_ds = train_ds.remove_columns(['id', 'title', 'context', 'question', 'answers', 'ans_text', 'input_texts'])

In [21]:
train_ds.set_format("torch")

In [22]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_ds, shuffle=True, batch_size=1)
eval_dataloader = DataLoader(train_ds, batch_size=1)

In [23]:
train_ds["input_ids"][0]

tensor([ 1525,    10,    16,     8,  1480,  5541,     7,  2625,    10,   493,
           63,   106,    75,   154,  3156,     7,   693,  8900,   965,    18,
         6936,   449,    41,    87,   115,    23,     2,   354,     2,    29,
            7,    15,     2,    87,    36,    15,    18,   476,  4170,    18,
         8735,    61,    41,  7473,  1600,  6464, 15465,    61,    19,    46,
          797,  7634,     6,     3, 21101,     6,  1368,  8211,    11, 15676,
            5, 12896,    11,  3279,    16,  8018,     6,  2514,     6,   255,
         3032,    16,   796,  8782,    11, 10410,  2259,     7,    38,     3,
            9,   861,     6,    11,  4659,    12, 10393,    16,     8,  1480,
         5541,     7,    38,   991,  7634,    13,   391,   184,   279,  3202,
           18, 10739, 19344,    63,    31,     7,  9364,     5, 19607,    26,
           57,   160,  2353,     6,  9762,    15,   210,  8900,   965,     6,
            8,   563,  1632,    80,    13,     8,   296,    31, 

In [24]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

In [25]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [26]:
# model.half()

In [27]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [28]:
!nvidia-smi

Fri May 27 15:01:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.23       Driver Version: 511.23       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   46C    P0    13W /  N/A |    979MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [29]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/3000 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 2.87 GiB already allocated; 0 bytes free; 2.87 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
metric = load_metric("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

In [ ]:
# next(iter(train_dataloader))

In [ ]:
# train_ds["input_texts"]

In [ ]:
tokenizer.decode(train_ds["input_ids"][0])

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["context"], padding="max_length", truncation=True)

validation_ds = validation_dataset.map(tokenize_function, batched=True)
validation_ds